In [ ]:
#!/env/bin/python

import init

import findspark
findspark.init()

import os
import pandas as pd
import numpy as np
from pyspark.sql import *
from pyspark import *
import pyspark.sql.functions as f
from timeit import default_timer as timer
import logging
from pyrecdp.data_processor import *
from pyrecdp.utils import *
from pyspark.sql.functions import pandas_udf, PandasUDFType


def process_data(df, proc, output_name):
    proc.spark.udf.registerJavaFunction("dummy","com.intel.recdp.SortStringArrayByFrequency")
    op_feature_add = FeatureAdd(cols={'tweet': "f.expr('dummy(text_tokens)')"}, op='inline')

    # execute
    proc.reset_ops([op_feature_add])
    t1 = timer()
    df = proc.transform(df, name=output_name, df_cnt=626242930)
    t2 = timer()
    print("process_data decode took %.3f" % (t2 - t1))

    return df

In [ ]:
path_prefix = "hdfs://"
current_path = "/recsys2021_0608_scalaudf_test/"
original_folder = "/recsys2021_0608/"

scala_udf_jars = "/mnt/nvme2/chendi/BlueWhale/recdp/ScalaProcessUtils/target/core-0.1.0-jar-with-dependencies.jar"

##### 1. Start spark and initialize data processor #####
t0 = timer()
spark = SparkSession.builder.master('yarn')\
    .appName("udf_column")\
    .config("spark.sql.broadcastTimeout", "7200")\
    .config("spark.cleaner.periodicGC.interval", "10min")\
    .config("spark.driver.extraClassPath", f"{scala_udf_jars}")\
    .config("spark.executor.extraClassPath", f"{scala_udf_jars}")\
    .config("spark.executor.memory", "20g")\
    .config("spark.executor.memoryOverhead", "6g")\
    .getOrCreate()


# 1.1 prepare dataFrames
# 1.2 create RecDP DataProcessor
proc = DataProcessor(spark, path_prefix,
                     current_path=current_path, shuffle_disk_capacity="1200GB")
spark.udf.registerJavaFunction("sortStringArrayByFrequency","com.intel.recdp.SortStringArrayByFrequency")

df = spark.read.parquet(path_prefix + original_folder)

df = df.select("reply_timestamp", 'tweet_id').groupby('tweet_id').agg(f.collect_list('reply_timestamp').alias('reply_timestamp'))
print(df.dtypes)
df = df.withColumn('sorted_reply_timestamp', f.expr('sortStringArrayByFrequency(reply_timestamp)'))
print(df.dtypes)

df.show()

# ===============================================
# decode tweet_tokens
#df = process_data(df, proc, output_name="scala_udf_test")

In [ ]:
df.show()